---
title: GLMM and Hierarchical Bayesian Models
subject: Methods
subtitle: Partial pooling for stable estimation of hospital-level SSI risk
exports: 
  - format: pdf
    template: curvenote
    # template: arxiv_two_column # requires abstract
  # - format: pdf+tex
keywords: [hierarchical Bayesian models, generalized linear mixed models, partial pooling, hospital profiling, surgical site infection, small-area estimation, healthcare quality]
---

In [ ]:
library(dplyr)
library(ggplot2)
library(plotly)
library(kableExtra)
library(IRdisplay)
library(rjags)

# Load data
dat <- read.csv(
  "data/ca_ssi_adult_odp_2024.csv",
  header = TRUE,
  fileEncoding = "latin1"
)

# Restrict to colon procedures
colon_fac <- dat |>
  filter(Operative_Procedure == "Colon surgery") |>
  select(
    County,
    Facility_ID,
    Facility_Name,
    Hospital_Category_RiskAdjustment,
    Facility_Type,
    Procedure_Count,
    Infections_Reported,
    Infections_Predicted,
    SIR,
    SIR_CI_95_Upper_Limit,
    SIR_CI_95_Lower_Limit,
    Comparison,
    Notes
  ) |>
  mutate(observed_rate_fac = Infections_Reported/Procedure_Count)

# Use ACH only and drop pediatric facilities
colon_fac_ach <- colon_fac |>
  filter(Hospital_Category_RiskAdjustment == "Acute Care Hospital",
         Facility_Type != "Pediatric") |>
  mutate(County_numfactor = as.numeric(factor(County)),
         Facility_ID_numfactor = as.numeric(factor(Facility_ID)),
         Facility_Type = factor(Facility_Type),
         No_Infections = Procedure_Count - Infections_Reported) |>
  select(-Hospital_Category_RiskAdjustment) |>
  # pooled county-level counts and rates
  group_by(County) |>
    mutate(
      county_procedures = sum(Procedure_Count, na.rm = TRUE),
      county_infections = sum(Infections_Reported, na.rm = TRUE),
      observed_rate_county = county_infections / county_procedures
    ) |>
  ungroup()

# State mean SSI rate
state_mean <- sum(colon_fac_ach$Infections_Reported) / 
  sum(colon_fac_ach$Procedure_Count)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows


Loading required package: coda

Linked to JAGS 4.3.2

Loaded modules: basemod,bugs

